In [ ]:
#pip install faker

In [1]:
from faker import Faker
from faker.providers import ssn
import pandas as pd
from datetime import datetime

# 1.1
1. Bazując na bibliotekach pandas (https://github.com/pandas-dev/pandas) i faker (https://github.com/joke2k/faker), a konkretnie metodzie ssn z klasy Provider dla lokalizacji pl_PL (https://github.com/joke2k/faker/blob/master/faker/providers/ssn/pl_PL/__init__.py#L33) zaimplementuj funkcję o nazwie generate_ssns, która zwraca obiekt typu Series z liczbą rekordów określoną parametrem wejściowym funkcji oraz wartościami reprezentującymi wygenerowane losowe numery PESEL. 

In [2]:
def generate_ssns(elements):
    # log time
    # start = datetime.now()
    # log time

    fake = Faker(locale='pl_PL')
    ssns_list = []
    for element in range(elements):
        value = fake.ssn()
        ssns_list.append(value)

    ssns_series = pd.Series(ssns_list)

    return ssns_series

In [10]:
ds = generate_ssns(1)
print(type(ds),ds)

<class 'pandas.core.series.Series'> 0    21302527680
dtype: object


# 1.2
2. Zaimplementuj funkcję generate_unique_ssns (w dowolny sposób), która zwraca obiekt typu Series z liczbą rekordów określoną parametrem wejściowym funkcji oraz wartościami reprezentującymi wygenerowane losowe i unikalne (tylko w ramach zwracanej kolekcji) numery PESEL właściwe dla osób o płci (żeńska/męska) oraz urodzonych w przedziale dat (od-do) określonymi również parametrami wejściowymi tej funkcji. 

In [3]:
def is_valid_date_format(date_str):
    try:
        datetime.strptime(date_str, '%Y-%m-%d')
    except ValueError:
        raise ValueError("Incorrect data format, should be YYYY-MM-DD")


def is_valid_date(year, month, day):
    try:
        datetime(year, month, day)
    except ValueError:
        raise ValueError("Incorrect date, months 1-12,days 1-31")


def get_date_from_ssn(ssn_str):
    true_ssn_day = int(ssn_str[4:6])
    month_to_decode = int(ssn_str[2:4])
    true_ssn_month = month_to_decode % 20

    year_to_decode = int(ssn_str[0:2])
    if 1 <= month_to_decode <= 12:
        true_ssn_year = 1900 + year_to_decode
    elif 1 + 20 <= month_to_decode <= 12 + 20:
        true_ssn_year = 2000 + year_to_decode
    elif 1 + 40 <= month_to_decode < 12 + 40:
        true_ssn_year = 2100 + year_to_decode
    elif 1 + 60 <= month_to_decode < 12 + 60:
        true_ssn_year = 2200 + year_to_decode
    elif 1 + 80 <= month_to_decode < 12 + 80:
        true_ssn_year = 1800 + year_to_decode
    else:
        true_ssn_year = 0
    is_valid_date(true_ssn_year, true_ssn_month, true_ssn_day)
    true_ssn_date = datetime(true_ssn_year, true_ssn_month, true_ssn_day)
    return true_ssn_date

def generate_unique_ssns(elements, sex, in_start_dte, in_end_dte):
    ssns_list = []
    while len(ssns_list) + 1 <= elements:

        # ssn_str = fake.ssn()
        ssn_str = generate_ssns(1).tolist()[0]

        if ssn_str not in ssns_list:
            start_dte = datetime.strptime(in_start_dte, '%Y-%m-%d')
            is_valid_date_format(in_start_dte)
            end_dte = datetime.strptime(in_end_dte, '%Y-%m-%d')
            is_valid_date_format(in_end_dte)

            true_ssn_date = get_date_from_ssn(ssn_str)

            if start_dte <= true_ssn_date <= end_dte:
                if sex == 'F' and int(ssn_str[9]) % 2 == 0:
                    ssns_list.append(ssn_str)
                elif sex == 'M' and int(ssn_str[9]) % 2 != 0:
                    ssns_list.append(ssn_str)

    ssns_series = pd.Series(ssns_list)

    return ssns_series

# 1.3
3.     Następnie zaimplementuj wywołania funkcji generate_ssns oraz generate_unique_ssns dla 1 000, 10 000 i 100 000 rekordów, wskazując wybraną płeć oraz przedział dat urodzenia od 1990-01-01 do 1990-01-19. Dokonaj pomiaru i wyświetl czas trwania ich wykonania (osobno dla każdego wywołania każdej z tych dwóch funkcji). 

In [12]:
%%time
generate_ssns(1000)

Wall time: 77.1 ms


0      76032010191
1      02282633444
2      08311094457
3      06221415302
4      72022903485
          ...     
995    89101062014
996    91020736424
997    73060254928
998    09210570228
999    82092923500
Length: 1000, dtype: object

In [13]:
%%time
generate_ssns(10000)

Wall time: 347 ms


0       72012173236
1       98031281572
2       85071697757
3       21251479126
4       78111612863
           ...     
9995    08233028068
9996    83111876474
9997    04231934897
9998    18220344383
9999    78053069008
Length: 10000, dtype: object

In [14]:
%%time
generate_unique_ssns(1000, 'F', '1900-01-01', '1999-01-19')

Wall time: 2min 51s


0      86082379605
1      85011908244
2      96090425186
3      74022385766
4      72032852483
          ...     
995    96031011726
996    98091108406
997    97072963481
998    94082200306
999    98121732285
Length: 1000, dtype: object

In [5]:
%%time
generate_unique_ssns(10000, 'F', '1900-01-01', '1999-01-19')

Wall time: 27min 24s


0       91050599622
1       82060179825
2       85111547240
3       89041433107
4       77031176367
           ...     
9995    88032276000
9996    90100225085
9997    93092668649
9998    81112546684
9999    76071987487
Length: 10000, dtype: object

# 1.4
4. Zaimplementuj funkcję o nazwie validate_ssn, która na wejściu przyjmuje numer PESEL wraz z określeniem oczekiwanej płci (żeńska/męska/dowolna) i daty urodzenia (konkretna/określona lub dowolna), a na wyjściu zwraca informację o poprawności numeru PESEL. Wewnątrz funkcji zawrzyj logikę weryfikującą poprawność syntaktyczną numeru PESEL z uwzględnieniem informacji o oczekiwanej płci oraz dacie urodzenia. Następnie przetestuj działanie funkcji validate_ssn na przykładowych danych. 

In [4]:
def validate_ssn(ssn_str, sex, in_birth_dte):

    # TEST IS 11 INT
    if ssn_str.isnumeric() and len(ssn_str) == 11:

        # TEST LAST DIGIT
        check_digit_flag = ''
        weights_for_check_digit = [1, 3, 7, 9, 1, 3, 7, 9, 1, 3]
        sum_digit = 0

        for digits in range(0, 10):
            sum_digit += weights_for_check_digit[digits] * int(ssn_str[digits])
            # print(f"{sum_digit} =  {weights_for_check_digit[digits]} * {int(ssn_str[digits])}")

        check_digit = 10 - sum_digit % 10
        if check_digit == 10:
            check_digit = 0       
        if check_digit == int(ssn_str[10]):
            check_digit_flag = '1'
        else:
            check_digit_flag = '0'

        # TEST SEX
        check_sex_flag = ''
        if sex == 'dowolna':
            check_sex_flag = '?'
        else:
            if sex == 'F':
                if int(ssn_str[9]) % 2 == 0:
                    check_sex_flag = '1' # is female PESEL'
                else:
                    check_sex_flag = '0' # is not female PESEL'
            elif sex == 'M':
                if int(ssn_str[9]) % 2 != 0:
                    check_sex_flag = '1' # is male PESEL'
                else:
                    check_sex_flag = '0' # is not male PESEL'

        # TEST DATE OF BIRTH
        if in_birth_dte == 'dowolna':
            check_dte_flag = '?'
        else:
            is_valid_date_format(in_birth_dte)
            true_ssn_date = get_date_from_ssn(ssn_str)
            birth_dte = datetime.strptime(in_birth_dte, '%Y-%m-%d')
            if true_ssn_date == birth_dte:
                check_dte_flag = '1' # date of birth AS expected
            else:
                check_dte_flag = '0' # date of birth OTHER than expected
    # OUTPUT
        return f"PESEL:{ssn_str}|correct_pesel:{check_digit_flag}|{sex}:{check_sex_flag}|{in_birth_dte}:{check_dte_flag}"

    else:
        return f"PESEL:{ssn_str}|PESEL must have 11 digits"


In [5]:
# TESTY
print('\nvalidate_ssn -----------------2')
print(validate_ssn(generate_unique_ssns(1, 'F', '1900-01-01', '2099-12-31')[0], 'dowolna', 'dowolna'))
print(validate_ssn(generate_unique_ssns(1, 'M', '1900-01-01', '2099-12-31')[0], 'dowolna', 'dowolna'))
print(validate_ssn(generate_unique_ssns(1, 'F', '1900-01-01', '1999-12-31')[0], 'F', '1999-04-30'))
print(validate_ssn(generate_unique_ssns(1, 'M', '1900-01-01', '1999-12-31')[0], 'F', '1999-04-30'))
print(validate_ssn(generate_unique_ssns(1, 'M', '2000-01-01', '2099-12-31')[0], 'M', '1999-04-30'))
print(validate_ssn(generate_unique_ssns(1, 'F', '2000-01-01', '2099-12-31')[0], 'M', '1999-04-30'))


validate_ssn -----------------2
PESEL:20302284885|correct_pesel:1|dowolna:?|dowolna:?
PESEL:13301488176|correct_pesel:1|dowolna:?|dowolna:?
PESEL:73112061944|correct_pesel:1|F:1|1999-04-30:0
PESEL:89011586994|correct_pesel:1|F:0|1999-04-30:0
PESEL:01312428977|correct_pesel:1|M:1|1999-04-30:0
PESEL:08310945549|correct_pesel:1|M:0|1999-04-30:0


In [9]:
print('\n\nvalidate_ssn -----------------3')
test_set0 = ['123456789012', '1234567890', '', 'abcdefghijk','1ikl..+./']
test_set1 = ['10121378901', '93120611479', '08292960201', '05322226924']
test_set2 = ['93120611478', '08292960208', '05322226925']
test_set3 = ['06222487465'] 
test_set4 = ['93031378554'] 
test_set5 = ['24822907765', '00010104170', '99323108267', '45422805035', '00610107270'] 


print('\ntest_set0')
for test in test_set0:
    print(validate_ssn(test, 'F', '1986-04-30')+'exp PESEL must have 11 digits')
    
print('\ntest_set1')
for test in test_set1:
    print(validate_ssn(test, 'F', '1986-04-30')+'exp correct_pesel:0')
          
print('\ntest_set2')         
for test in test_set2:
    print(validate_ssn(test, 'F', '2006-06-14')+'exp correct_pesel:1')

# exp correct_pesel:1          
print('\ntest_set3')
for test in test_set3:
    print(validate_ssn(test, 'F', '2006-02-24')+'exp F:1,2006-02-24:1')
    print(validate_ssn(test, 'M', '2106-02-23')+'exp F:0,2006-02-24:0')
    print(validate_ssn(test, 'F', '2006-02-25')+'exp F:1,2006-02-24:0')
    print(validate_ssn(test, 'dowolna', '2006-02-24')+'exp:dowolna:?,2006-02-24:1')
    print(validate_ssn(test, 'M', 'dowolna')+'exp M:0,dowolna:?')
    
# exp correct_pesel:1
print('\ntest_set4')
for test in test_set4:
    print(validate_ssn(test, 'M', '1993-03-13')+'exp M:1,1993-03-13:1')
    print(validate_ssn(test, 'F', '1993-03-13')+'exp F:0,1993-03-13:1')
    print(validate_ssn(test, 'F', '2993-03-13')+'exp F:0,1993-03-13:0')
    print(validate_ssn(test, 'dowolna', '1999-03-12')+'exp dowolna:?,19903-03-13:0')    
    print(validate_ssn(test, 'M', 'dowolna')+'exp M:1,dowolna:?')
    
# exp correct_pesel:1
print('\ntest_set5')
print(validate_ssn(test_set5[0], 'F', '1824-02-29')+'exp F:1,1824-02-29:1')
print(validate_ssn(test_set5[1], 'M', '1900-01-01')+'exp M:1,1900-01-01:1')
print(validate_ssn(test_set5[2], 'M', '2099-12-31')+'exp M:0,2099-12-31:1')
print(validate_ssn(test_set5[3], 'M', '2145-02-28')+'exp M:1,2145-02-28:1')
print(validate_ssn(test_set5[4], 'F', '2200-01-01')+'exp F:0,2200-01-01:1')

    




validate_ssn -----------------3

test_set0
PESEL:123456789012|PESEL must have 11 digitsexp PESEL must have 11 digits
PESEL:1234567890|PESEL must have 11 digitsexp PESEL must have 11 digits
PESEL:|PESEL must have 11 digitsexp PESEL must have 11 digits
PESEL:abcdefghijk|PESEL must have 11 digitsexp PESEL must have 11 digits
PESEL:1ikl..+./|PESEL must have 11 digitsexp PESEL must have 11 digits

test_set1
PESEL:10121378901|correct_pesel:0|F:1|1986-04-30:0exp correct_pesel:0
PESEL:93120611479|correct_pesel:0|F:0|1986-04-30:0exp correct_pesel:0
PESEL:08292960201|correct_pesel:0|F:1|1986-04-30:0exp correct_pesel:0
PESEL:05322226924|correct_pesel:0|F:1|1986-04-30:0exp correct_pesel:0

test_set2
PESEL:93120611478|correct_pesel:1|F:0|2006-06-14:0exp correct_pesel:1
PESEL:08292960208|correct_pesel:1|F:1|2006-06-14:0exp correct_pesel:1
PESEL:05322226925|correct_pesel:1|F:1|2006-06-14:0exp correct_pesel:1

test_set3
PESEL:06222487465|correct_pesel:1|F:1|2006-02-24:1exp F:1,2006-02-24:1
PESEL:062

In [9]:
print('\nis_valid_date_format -----------------------')
# exp ValueError("Incorrect data format, should be YYYY-MM-DD")
print(is_valid_date_format('2021-31-01')) 
print(is_valid_date(1999, 4, 31))


is_valid_date_format -----------------------


ValueError: Incorrect data format, should be YYYY-MM-DD

In [10]:
print('\nget_date_from_ssn -----------------------')
# exp ValueError("Incorrect date, it is not a date")
print(get_date_from_ssn('06224487465')) 


get_date_from_ssn -----------------------


ValueError: Incorrect date, months 1-12,days 1-31